## 📌 Task 4: Churn Prediction Model

The objective of this task is to build and evaluate machine learning models
to predict customer churn in a telecommunications company.

Multiple classification algorithms are implemented and compared using
appropriate evaluation metrics. The best-performing model is then selected
for further analysis and interpretation.


### 1️⃣ Import Required Libraries

We import the necessary Python libraries for:
- Data handling
- Model training
- Model evaluation
- Hyperparameter tuning


In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score
)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore")


### 2️⃣ Load the Cleaned Dataset

The cleaned and preprocessed dataset generated in **Task 1**
is loaded here for model building.


In [5]:
data = pd.read_csv(
    "../Task-1_Data_Preparation/dataset/Telco_Customer_Churn_Dataset_cleaned.csv"
)

data.head()


,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes
0,7590-VHVEG,0,1,29.85,29.85,False,True,False,False,True,...,False,False,False,False,False,True,False,True,False,False
1,5575-GNVDE,0,34,56.95,1889.50,True,False,False,True,False,...,False,False,False,True,False,False,False,False,True,False
2,3668-QPYBK,0,2,53.85,108.15,True,False,False,True,False,...,False,False,False,False,False,True,False,False,True,True
3,7795-CFOCW,0,45,42.30,1840.75,True,False,False,False,True,...,False,False,False,True,False,False,False,False,False,False
4,9237-HQITU,0,2,70.70,151.65,False,False,False,True,False,...,False,False,False,False,False,True,False,True,False,True


### 3️⃣ Define Features and Target Variable

- **Target Variable:** `Churn_Yes`
- **Features:** All remaining columns

This target represents whether a customer has churned (`1`) or not (`0`).


In [9]:
X = data.drop(["Churn_Yes", "customerID"], axis=1)
y = data["Churn_Yes"]

### 4️⃣ Train–Test Split

The dataset is split into training and testing sets using **stratified sampling**
to preserve the original churn distribution.

- Training set: 80%
- Testing set: 20%


In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


### 5️⃣ Feature Scaling

Feature scaling is applied **only to Logistic Regression**, as it is sensitive
to feature magnitudes.

Tree-based models do not require feature scaling.


In [11]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### 6️⃣ Model Evaluation Strategy

Multiple classification models are trained and evaluated using:

- Accuracy
- Precision
- Recall
- F1-score

Since customer churn datasets are typically **imbalanced**, **F1-score**
is prioritized during model comparison.


### 7️⃣ Logistic Regression Model

Logistic Regression is used as a baseline model due to its simplicity
and interpretability.

Class imbalance is handled using `class_weight="balanced"`.


In [13]:
log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)

log_reg.fit(X_train_scaled, y_train)

y_pred_lr = log_reg.predict(X_test_scaled)

lr_metrics = {
    "Model": "Logistic Regression",
    "Accuracy": accuracy_score(y_test, y_pred_lr),
    "Precision": precision_score(y_test, y_pred_lr),
    "Recall": recall_score(y_test, y_pred_lr),
    "F1-Score": f1_score(y_test, y_pred_lr)
}

lr_metrics


{'Model': 'Logistic Regression',
 'Accuracy': 0.7395315826827538,
 'Precision': 0.5060240963855421,
 'Recall': 0.786096256684492,
 'F1-Score': 0.6157068062827226}

### 8️⃣ Decision Tree Classifier

Decision Tree is a rule-based model capable of capturing
non-linear relationships in the data.

Class imbalance is addressed using balanced class weights.


In [14]:
dt = DecisionTreeClassifier(
    random_state=42,
    class_weight="balanced"
)

dt.fit(X_train, y_train)

y_pred_dt = dt.predict(X_test)

dt_metrics = {
    "Model": "Decision Tree",
    "Accuracy": accuracy_score(y_test, y_pred_dt),
    "Precision": precision_score(y_test, y_pred_dt),
    "Recall": recall_score(y_test, y_pred_dt),
    "F1-Score": f1_score(y_test, y_pred_dt)
}

dt_metrics


{'Model': 'Decision Tree',
 'Accuracy': 0.7310149041873669,
 'Precision': 0.49318801089918257,
 'Recall': 0.4839572192513369,
 'F1-Score': 0.4885290148448043}

### 9️⃣ Random Forest Classifier

Random Forest is an ensemble learning method that combines multiple
decision trees to improve performance and reduce overfitting.

Hyperparameter tuning is performed using **GridSearchCV**
with **F1-score** as the optimization metric.


In [15]:
rf = RandomForestClassifier(
    random_state=42,
    class_weight="balanced"
)

param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2]
}

grid_rf = GridSearchCV(
    rf,
    param_grid,
    cv=3,
    scoring="f1",
    n_jobs=-1
)

grid_rf.fit(X_train, y_train)

best_rf = grid_rf.best_estimator_


### 🔍 Random Forest Evaluation

The best Random Forest model is evaluated using standard classification metrics
along with **ROC-AUC**, which measures the model’s ability to distinguish
between churned and non-churned customers.


In [16]:
y_pred_rf = best_rf.predict(X_test)
y_prob_rf = best_rf.predict_proba(X_test)[:, 1]

rf_metrics = {
    "Model": "Random Forest",
    "Accuracy": accuracy_score(y_test, y_pred_rf),
    "Precision": precision_score(y_test, y_pred_rf),
    "Recall": recall_score(y_test, y_pred_rf),
    "F1-Score": f1_score(y_test, y_pred_rf),
    "ROC-AUC": roc_auc_score(y_test, y_prob_rf)
}

rf_metrics


{'Model': 'Random Forest',
 'Accuracy': 0.772888573456352,
 'Precision': 0.552734375,
 'Recall': 0.7566844919786097,
 'F1-Score': 0.6388261851015802,
 'ROC-AUC': 0.8426489963574362}

### 📊 Model Comparison

The performance of all trained models is summarized below.
Metric values are rounded for better readability.


In [17]:
results = pd.DataFrame([lr_metrics, dt_metrics, rf_metrics])
results = results.round(3)
results


,Model,Accuracy,Precision,Recall,F1-Score,ROC-AUC
0,Logistic Regression,0.740,0.506,0.786,0.616,NaN
1,Decision Tree,0.731,0.493,0.484,0.489,NaN
2,Random Forest,0.773,0.553,0.757,0.639,0.843


ROC-AUC was computed only for the Random Forest model, as it provides stable probability estimates suitable for ranking-based evaluation.

### ✅ Final Conclusion

Three classification models—Logistic Regression, Decision Tree, and Random Forest—
were trained and evaluated for customer churn prediction.

Among the models, Random Forest demonstrated superior performance across
precision, recall, F1-score, and ROC-AUC, making it the most reliable model
for identifying at-risk customers.

Therefore, **Random Forest** was selected as the final churn prediction model
for further evaluation, interpretation, and business recommendations.
